In [1]:
#load in the used packages
import pandas as pd
import numpy as np
import requests, sys
from collections import Counter

In [2]:
# load the needed databases or donload them
kegg_server = "http://rest.kegg.jp/"
kegg_protein_list = "list/hsa"

kegg_request = requests.get(kegg_server +kegg_protein_list)
if not kegg_request.ok:
    kegg_request.raise_for_status()
    sys.exit()
ensmbl_to_kegg = pd.DataFrame([x.split("\t") for x in kegg_request.text.split("\n")], columns=["ID","names"])
ensmbl_to_kegg = ensmbl_to_kegg.set_index("ID").dropna()
ensmbl_to_kegg = ensmbl_to_kegg.names.apply(lambda x: x.split("; ")[0])
ensmbl_to_kegg = ensmbl_to_kegg.apply(lambda x: x.split(", "))


kegg_server = "http://rest.kegg.jp/"
kegg_protein_pathway = "/link/hsa/pathway"

pathway_request = requests.get(kegg_server +kegg_protein_pathway)
if not pathway_request.ok:
    pathway_request.raise_for_status()
    sys.exit()
path_df = pd.DataFrame([x.split("\t") for x in pathway_request.text.split("\n")], columns=["pathway", "gene_id"])


In [3]:
#Function to retrieve the pathways for each part of the a chromosome.
col_names = ['end', 'biotype', 'strand', 'feature_type', 'id', 'version', 'seq_region_name', 'start',
             'logic_name', 'assembly_name', 'description', 'gene_id', 'source', 'external_name']
def get_pathways(Ch,start,end):
    kegg_id = []
    server = "https://rest.ensembl.org"
    ext = "/overlap/region/human/{0}:{1}-{2}?feature=gene;biotype=protein_coding;a=ncbi36".format(Ch, start,end)
    r = requests.get(server +ext, headers={ "Content-Type" : "application/json"})
    if not r.ok:
        print(f"Error at CH {Ch} from {start} till {end}")
        return []
    #print(pd.DataFrame(r.json()))
    genes = pd.DataFrame(r.json(), columns=col_names).external_name
    for id in list(genes):
        kegg_id += list(ensmbl_to_kegg[ensmbl_to_kegg.map(set([id]).issubset)].index)
    data = path_df[path_df.gene_id.apply(lambda x: x in kegg_id)].pathway.tolist()
    return data

In [227]:
#load the data:
data = pd.read_csv(sys.path[1]+"/raw_data/train_call.txt",sep="\t")
classes = pd.read_csv(sys.path[1]+"/raw_data/train_clinical.txt", sep="\t")
index = data.transpose().index[:4]
classes

,Sample,Subgroup
0,Array.129,HER2+
1,Array.34,HR+
2,Array.67,HR+
3,Array.24,Triple Neg
4,Array.22,Triple Neg
...,...,...
95,Array.10,HER2+
96,Array.123,HR+
97,Array.100,HR+
98,Array.134,HR+


In [228]:
data = data.transpose().merge(classes, how="left", right_on="Sample",left_index=True)
data = data.set_index("Sample").dropna(axis=1)
data

,0,1,2,3,4,5,6,7,8,9,...,1342,1343,1344,1345,1346,1347,1348,1349,1350,1351
Sample,,,,,,,,,,,,,,,,,,,,,
Chromosome,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0
Start,2927.0,85022.0,370546.0,471671.0,792533.0,912799.0,1271190.0,1676445.0,1738295.0,2481927.0,...,58432737.0,60125080.0,63282635.0,64146695.0,66018580.0,66455837.0,67988973.0,68119528.0,71007045.0,72351591.0
End,43870.0,216735.0,372295.0,786483.0,907406.0,1266212.0,1590570.0,1703748.0,2477597.0,2562342.0,...,60103148.0,63239913.0,64126023.0,66004965.0,66423604.0,67974175.0,68108381.0,71002959.0,72336589.0,73557841.0
Nclone,3.0,4.0,4.0,5.0,13.0,96.0,45.0,4.0,104.0,14.0,...,84.0,244.0,83.0,110.0,17.0,100.0,14.0,324.0,156.0,111.0
Array.129,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Array.10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
Array.123,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Array.100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [192]:
topInfo = data.iloc[:4].transpose().iloc[:-2].transpose()

topInfo = topInfo.set_index(index).transpose()
topInfo = topInfo.astype(int)
topInfo = topInfo.transpose()
topInfo

,0,1,2,3,4,5,6,7,8,9,...,1340,1341,1342,1343,1344,1345,1346,1347,1348,1349
Chromosome,1,1,1,1,1,1,1,1,1,1,...,10,10,10,10,10,10,10,10,10,10
Start,2927,85022,370546,471671,792533,912799,1271190,1676445,1738295,2481927,...,55351133,55795426,58432737,60125080,63282635,64146695,66018580,66455837,67988973,68119528
End,43870,216735,372295,786483,907406,1266212,1590570,1703748,2477597,2562342,...,55782605,58408695,60103148,63239913,64126023,66004965,66423604,67974175,68108381,71002959
Nclone,3,4,4,5,13,96,45,4,104,14,...,48,154,84,244,83,110,17,100,14,324


,Sample,Subgroup
0.0,Array.129,HER2+
1.0,Array.34,HR+
2.0,Array.67,HR+
3.0,Array.24,Triple Neg
4.0,Array.22,Triple Neg
...,...,...
95.0,Array.10,HER2+
96.0,Array.123,HR+
97.0,Array.100,HR+
98.0,Array.134,HR+


In [193]:
data = data.iloc[4:].transpose().iloc[:-1].transpose()
data

,0,1,2,3,4,5,6,7,8,9,...,1341,1342,1343,1344,1345,1346,1347,1348,1349,1350
Sample,,,,,,,,,,,,,,,,,,,,,
Array.129,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Array.34,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
Array.67,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Array.24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Array.22,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Array.10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
Array.123,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Array.100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [290]:
#print(topInfo)
pathway_cluster = pd.DataFrame(columns=["Chromosome","Start","End"].append(path_df.pathway.unique()[:-1]))
max_seq_range = 5000000-1
print(pathway_cluster)
for i in topInfo.transpose().index:
#for i in range(10):
    full_data = []
    #print(topInfo.transpose().iloc[i])
    dataPoint = topInfo.transpose().iloc[i]
    #print(dataPoint[1],dataPoint[2] )
    Ch = dataPoint[0]
    seq_range = range(np.math.ceil((dataPoint[2] - dataPoint[1])// max_seq_range)+1)
    for x in seq_range:
        if x != seq_range[-1]:
            start = dataPoint[1]+(max_seq_range*x)
            end = dataPoint[1]+(max_seq_range*(x+1))
        elif x == seq_range[-1]:
            start = dataPoint[1]+(max_seq_range*x)
            end = dataPoint[1]+(max_seq_range*(x))+(dataPoint[2] - dataPoint[1])% max_seq_range
        data_test = get_pathways(Ch, start, end)
        #print(len(data_test), len(full_data))
        full_data+=data_test
    #print(Counter(full_data))
    full_data = {"Chromosome":Ch,"Start":dataPoint[1],"End":dataPoint[2], **Counter(full_data)}
    #full_data = pd.concat([dataPoint,full_data])

    pathway_cluster = pathway_cluster.append(full_data, ignore_index=True)
    #print(pathway_cluster)

Empty DataFrame
Columns: []
Index: []
Error at CH 2 from 242430070 till 242784096
Error at CH 3 from 199333505 till 199383479
Error at CH 4 from 190887518 till 191025356
Error at CH 4 from 191037678 till 191046020
Error at CH 4 from 191062719 till 191134340
Error at CH 4 from 191144773 till 191165971
Error at CH 4 from 191191940 till 191311992
Error at CH 6 from 170856656 till 170943103
Error at CH 8 from 145962567 till 146264846


In [296]:
t1 = topInfo.iloc[:3].transpose()
t2 = pathway_cluster.transpose().iloc[:3].transpose().astype(int)

print(t2.reindex(columns=list(t1)).compare(t1, ))


['Chromosome', 'Start', 'End']
Empty DataFrame
Columns: []
Index: []


In [297]:
data.dropna(axis=1)


,0,1,2,3,4,5,6,7,8,9,...,1342,1343,1344,1345,1346,1347,1348,1349,1350,1351
Sample,,,,,,,,,,,,,,,,,,,,,
Chromosome,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0
Start,2927.0,85022.0,370546.0,471671.0,792533.0,912799.0,1271190.0,1676445.0,1738295.0,2481927.0,...,58432737.0,60125080.0,63282635.0,64146695.0,66018580.0,66455837.0,67988973.0,68119528.0,71007045.0,72351591.0
End,43870.0,216735.0,372295.0,786483.0,907406.0,1266212.0,1590570.0,1703748.0,2477597.0,2562342.0,...,60103148.0,63239913.0,64126023.0,66004965.0,66423604.0,67974175.0,68108381.0,71002959.0,72336589.0,73557841.0
Nclone,3.0,4.0,4.0,5.0,13.0,96.0,45.0,4.0,104.0,14.0,...,84.0,244.0,83.0,110.0,17.0,100.0,14.0,324.0,156.0,111.0
Array.129,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Array.10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
Array.123,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Array.100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [294]:
#dotprosuct the data to retrieve the sumdata per
pathway_data = data.iloc[4:].to_numpy() @ pathway_cluster.fillna(0)
final_data = pd.concat([classes.reset_index().Subgroup, pathway_data.transpose()], ignore_index=True, axis=1)
final_data.columns = pathway_cluster.columns.insert(0,"Subgroup")
final_data= final_data.set_index("Subgroup")

ValueError: shapes (100, 1352) and (1350, 344) not aligned

In [ ]:
final_data.groupby("Subgroup").mean()

In [ ]:
final_data.to_csv(sys.path[1]+"pathway_data.csv")